In [ ]:
import pandas as pd
import sys
import requests

from tqdm.notebook import tnrange
from datetime import datetime

# Import utility functions
sys.path.insert(0, r'c:\Users\joneh\master_thesis\src')
from main_utils import *
from db_utils import *

### Functions

In [ ]:
def clean_date(date_str: str) -> datetime:
    date_time_obj = datetime.strptime(date_str.replace('Z', '+00:00'), "%Y-%m-%dT%H:%M:%S%z")
    return date_time_obj

def query_url(query: str, page: int, API_KEY: str) -> str:
    q = query.replace(' AND ', '%20AND%20')
    URL = f'https://content.guardianapis.com/search?from-date=2003-12-31&to-date=2023-12-31&order-by=newest&page={page}&page-size=200&q={q}&api-key={API_KEY}'
    return URL


### Data retrieval - The Guardian API

In [ ]:
# retrieve API KEY from file
with open('API_keys/TG_API_KEY.txt', 'r') as file:
    API_KEY = file.read()

# query
query = 'Natural AND Gas'

archive = []

first_page = requests.get(query_url(query, 1, API_KEY)).json()
pages = first_page['response']['pages']
archive.extend(first_page['response']['results'])

for page in tnrange(2, pages + 1):
    URL = query_url(query, page, API_KEY)
    response = requests.get(URL).json()

    if response['response']['status'] != 'ok':
        print(f'Error at page {page}')
        continue
        
    article_list = response['response']['results']

    archive.extend(article_list)

df_data = pd.DataFrame(archive)


### add tags

In [ ]:
df = df_data.copy()
# add datetime column
df['datetime'] = df['webPublicationDate'].apply(clean_date)

# add source tag
df['source'] = 'TG'

# add query
df['query'] = query.replace(" ", "")

df.drop(columns=['type', 'sectionId', 'sectionName', 'webPublicationDate', 'apiUrl', 'isHosted', 'pillarId', 'pillarName'], inplace=True)

df.rename(columns={'id': 'article_id', 'webTitle': 'headline', 'webUrl': 'web_url'}, inplace=True)

df = df[['datetime', 'article_id', 'headline', 'web_url', 'source', 'query']]

display(df.head(3))


### Save dataset to folder

In [ ]:

# Enter filename here:
file_name = f'TG_{query.replace(" ", "")}.csv'
# Enter relative path for saving the file:
relative_path = 'data/news'

display(df.head())

df.to_csv(save_path(relative_path, file_name), index=False)

### Save to database

In [ ]:
news_db_commit(df, 'news')
db_info(show_table=True)